#Transformations du bronze en Silver

Ce notebook effectue des transformations sur les données du lakehouse Bronze et enregistre les données de transformation dans le lakehouse Silver.

In [ ]:
# Importation des fonctions nécessaires pour le projet
from pyspark.sql.functions import round, col, dayofmonth, month, year, to_date, quarter, substring, when, regexp_replace

In [ ]:
# Définis le chemin d'accès à la table wind_power_production dans Lakehouse Bronze
bronze_table_path = "abfss://WindPowerGeneration@onelake.dfs.fabric.microsoft.com/LH_Bronze.Lakehouse/Tables/wind_power_production"

# Charge la table wind_power_production dans un DataFrame spark
df = spark.read.format("delta").load(bronze_table_path)

In [ ]:
# Nettoyage et enrichissement des données
df_transformed = (df
    .withColumn("wind_speed", round(col("wind_speed"), 2))
    .withColumn("energy_produced", round(col("energy_produced"), 2))
    .withColumn("day", dayofmonth(col("date")))
    .withColumn("month", month(col("date")))
    .withColumn("quarter", quarter(col("date")))
    .withColumn("year", year(col("date")))
    .withColumn("time", regexp_replace(col("time"), "-", ":"))
    .withColumn("hour_of_day", substring(col("time"), 1, 2).cast("int"))
    .withColumn("minute_of_hour", substring(col("time"), 4, 2).cast("int"))
    .withColumn("second_of_minute", substring(col("time"), 7, 2).cast("int"))
    .withColumn("time_period", when((col("hour_of_day") >= 5 ) & (col("hour_of_day") < 12), "Morning")
                                .when((col("hour_of_day") >= 12 ) & (col("hour_of_day") < 17), "Afternoon")
                                .when((col("hour_of_day") >= 17 ) & (col("hour_of_day") < 21), "Evening")
                                .otherwise("Night")
    )
)

In [ ]:
#Spécifie le chemin vers la table wind_power_production située dans le Lakehouse (niveau Silver)
silver_table_path = "abfss://WindPowerGeneration@onelake.dfs.fabric.microsoft.com/LH_Silver.Lakehouse/Tables/wind_power_production"

#Sauvegarde la version transformée de la table wind_power_production dans le Lakehouse (niveau Silver)
df_transformed.write.format("delta").mode("overwrite").save(silver_table_path)